# 葡萄糖生成自由能对pH和pMg的依赖性分析

本笔记本分析葡萄糖在不同pH和pMg条件下的生成自由能变化。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from equilibrator_api import ComponentContribution, Q_
import pandas as pd
import seaborn as sns

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 初始化ComponentContribution
cc = ComponentContribution()

In [ ]:
def calculate_glucose_dgf_prime(p_h, p_mg, t=298.15, ionic_strength=0.25):
    """
    计算葡萄糖在指定pH和pMg条件下的生成自由能
    
    参数:
    p_h: pH值
    p_mg: pMg值
    t: 温度 (K)
    ionic_strength: 离子强度 (M)
    
    返回:
    formation_energy: 生成自由能 (kJ/mol)
    """
    try:
        # 获取葡萄糖化合物对象
        compound = cc.get_compound('glucose')
        
        # 计算在指定条件下的生成自由能
        transformed = compound.transform(
            p_h=Q_(p_h),
            p_mg=Q_(p_mg),
            ionic_strength=Q_(f'{ionic_strength}M'),
            temperature=Q_(f'{t}K')
        )
        
        # 返回生成自由能值 (kJ/mol)
        return transformed.value.m
    
    except Exception as e:
        print(f"计算出错: {e}")
        return np.nan

In [ ]:
# 定义pH和pMg范围
pH_range = np.linspace(4, 10, 20)
pMg_range = np.linspace(1, 14, 20)

# 创建网格
P_H, P_MG = np.meshgrid(pH_range, pMg_range)

# 计算对应的生成自由能
print("开始计算不同pH和pMg条件下的葡萄糖生成自由能...")
DG_F_PRIME = np.zeros_like(P_H)

for i in range(len(pMg_range)):
    for j in range(len(pH_range)):
        DG_F_PRIME[i, j] = calculate_glucose_dgf_prime(P_H[i, j], P_MG[i, j])
        
        # 显示进度
        if (i * len(pH_range) + j + 1) % 50 == 0:
            print(f"进度: {(i * len(pH_range) + j + 1) / (len(pMg_range) * len(pH_range)) * 100:.1f}%")

print("计算完成!")

In [ ]:
# 3D可视化
fig = plt.figure(figsize=(14, 5))

# 3D表面图
ax1 = fig.add_subplot(131, projection='3d')
surf = ax1.plot_surface(P_H, P_MG, DG_F_PRIME, cmap='viridis', alpha=0.8)
ax1.set_xlabel('pH')
ax1.set_ylabel('pMg')
ax1.set_zlabel('生成自由能 (kJ/mol)')
ax1.set_title('葡萄糖生成自由能随pH和pMg变化的3D图')

# 等高线图
ax2 = fig.add_subplot(132)
contour = ax2.contour(P_H, P_MG, DG_F_PRIME, levels=20)
ax2.clabel(contour, inline=True, fontsize=8)
ax2.set_xlabel('pH')
ax2.set_ylabel('pMg')
ax2.set_title('葡萄糖生成自由能等高线图')

# 热力图
ax3 = fig.add_subplot(133)
im = ax3.imshow(DG_F_PRIME, extent=[pH_range.min(), pH_range.max(), pMg_range.min(), pMg_range.max()], 
                aspect='auto', origin='lower', cmap='viridis')
ax3.set_xlabel('pH')
ax3.set_ylabel('pMg')
ax3.set_title('葡萄糖生成自由能热力图')
plt.colorbar(im, ax=ax3, label='生成自由能 (kJ/mol)')

plt.tight_layout()
plt.show()

In [ ]:
# 创建结果摘要
print(f"葡萄糖生成自由能范围:")
print(f"最小值: {np.nanmin(DG_F_PRIME):.2f} kJ/mol")
print(f"最大值: {np.nanmax(DG_F_PRIME):.2f} kJ/mol")
print(f"平均值: {np.nanmean(DG_F_PRIME):.2f} kJ/mol")
print(f"标准差: {np.nanstd(DG_F_PRIME):.2f} kJ/mol")

In [ ]:
# 保存数据到CSV
df = pd.DataFrame({
    'pH': P_H.flatten(),
    'pMg': P_MG.flatten(),
    'Delta_G_f_prime_kJ_per_mol': DG_F_PRIME.flatten()
})

df.to_csv('glucose_dgf_prime_ph_pmg.csv', index=False)
print("数据已保存到 glucose_dgf_prime_ph_pmg.csv")